In [1]:
import ee
from IPython.display import Image
import geemap
import pandas as pd

In [2]:
ee.Authenticate()
ee.Initialize(project='ee-wherediddavidgo')

In [3]:
import rssa_utils as utils

In [4]:
clBuffer = ee.FeatureCollection('projects/ee-wherediddavidgo/assets/s2_platte_centerlines_3xbuffer')
rCl = ee.Image('projects/ee-wherediddavidgo/assets/1px_cl_mosaic')
vCl = ee.FeatureCollection('projects/ee-wherediddavidgo/assets/s2_platte_centerlines')
# xSectionEnds = ee.FeatureCollection('projects/ee-wherediddavidgo/assets/platte_x_section_ends_200m')
# xLines = ee.FeatureCollection('projects/ee-wherediddavidgo/assets/platte_xlines_5xmean_200m')

pts = ee.FeatureCollection('projects/ee-wherediddavidgo/assets/platte_points')
watershed = ee.FeatureCollection('projects/ee-wherediddavidgo/assets/platte_watershed_boundary')
# classifier_table = utils.TRAIN_KNN()
# classifier_table = classifier_table.set_index('month')

big_ic = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 20)\
    .filterBounds(pts.geometry())

In [5]:
def EXTRACT_WIDTHS_FROM_IMAGE(scene):
    aoi = clBuffer.geometry().intersection(scene.geometry())

    # month = (scene.date().get('month'))
    
    # tile = ee.String('T').cat(scene.get('MGRS_TILE'))

    # knnClassifier = classifier_table.at[month, 'classifier']
    scene = utils.ADD_WATER_MASK(scene, aoi)
    scene = utils.EXTRACT_RIVER(scene, rCl, 1000, 333)

    scene = utils.UNPACK_SCL(scene)
    # scene = utils.ADD_CLOUD_MASK(scene)
    # scene = utils.ADD_SNOW_FLAG(scene)
    # scene = utils.CALC_HILLSHADOW(scene)

    widths = utils.CALCULATE_WIDTH(scene, pts.filterBounds(scene.geometry()))

    cpp = scene.get('CLOUDY_PIXEL_PERCENTAGE')
    dt = scene.date()

    
    def add_info(f):
        return f.set({'scene_cloudy_pixel_percentage': cpp,
                      'scene_date': dt})
    
    widths = widths.map(add_info)
    

    return widths

extract widths per tile per year

In [6]:
props = ['system:index', 'img_id', 'MLength', 'any', 'cloud_mask', 'count', 'endsInWater', 'endsOverEdge', 'iindex', 'river_mask', 'scene_cloudy_pixel_percentage', 'scene_date', 'snow_mask', 'width']
for year in range(2019, 2025):
    for month in range(1, 13):
        temp_ic = big_ic\
            .filter(ee.Filter.calendarRange(month, None, 'month'))\
            .filter(ee.Filter.calendarRange(year, None, 'year'))
        
        widths = temp_ic.map(EXTRACT_WIDTHS_FROM_IMAGE).flatten().select(props)
            
        task = ee.batch.Export.table.toDrive(**{
            'collection': widths,
            'folder': 'rwc_exports',
            'fileNamePrefix': f'{year}_{month}_widths',
            'description': f'{year}_{month}_widths'
        })

        task.start()
            
        print(f'year: {year}, month: {month}, n images: {temp_ic.size().getInfo()}')

year: 2019, month: 1, n images: 115
year: 2019, month: 2, n images: 82
year: 2019, month: 3, n images: 146
year: 2019, month: 4, n images: 83
year: 2019, month: 5, n images: 91
year: 2019, month: 6, n images: 172
year: 2019, month: 7, n images: 222
year: 2019, month: 8, n images: 192
year: 2019, month: 9, n images: 192
year: 2019, month: 10, n images: 198
year: 2019, month: 11, n images: 129
year: 2019, month: 12, n images: 119
year: 2020, month: 1, n images: 103
year: 2020, month: 2, n images: 122
year: 2020, month: 3, n images: 138
year: 2020, month: 4, n images: 116
year: 2020, month: 5, n images: 95
year: 2020, month: 6, n images: 194
year: 2020, month: 7, n images: 230


EEException: User memory limit exceeded.